In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time, os
import sys

actions = ['pause', 'play', 'volume_up', 'volume_down']
seq_length = 30 #시계열의 총 길이= 하나의 데이터를 추출할 때 몇 프레임을 묶어서 하나의 시퀀스 데이터로 만들것인지를 결정
secs_for_action = 30 #한 액션을 수집하는 시간

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,# 감지하려는 손의 개수를 나타냄
    min_detection_confidence=0.5,# 손을 감지히는 최소 신뢰도
    min_tracking_confidence=0.5)# 손을 추적하는 최소 신뢰도

cap = cv2.VideoCapture(0) # 비디어 캡어 객체를 생성, 0은 웹캠을 사용함을 의미

created_time = int(time.time())# 파일 이름을 시간으로 저장하기 위해.
os.makedirs('dataset', exist_ok=True)# dataset 폴더를 만듬. exist_ok를 사용해 폴더 존재 여부에 대한 에러 제거

while cap.isOpened():#캠의 상태여부에 따라 반복
    for idx, action in enumerate(actions):# actions의 리스트 항먹을 action에서 받게 되고 해당 인덱스를 idx변수로 받아 순회하고 된다.
        data = []

        ret, img = cap.read()#ret은 프레임의 읽어드리는 성공 여두에 따라 불린형으로 들어옴.img의 경우 numpy배열로 표현됨.

        img = cv2.flip(img, 1)#캠으로 입력받은 좌우반전된 img를 수평 뱡향으로 뒤집어 줌으로써 기본 좌우 상태로 돌리는 방법이다.

       #프로그램 시작 시 잠깐은 텀? 준비? 과정을 만들어 준거임 
        #puttext 함수를 통해 이미지 위에 waiting~action...을 표시한다.
        cv2.putText(img, f'Waiting for collecting {action.upper()} action...', org=(10, 30), 
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        cv2.imshow('img', img)
        cv2.waitKey(3000)

        start_time = time.time()
        #이제 진짜 데이터 수집 과정임
        
        while time.time() - start_time < secs_for_action:
            ret, img = cap.read()

            img = cv2.flip(img, 1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)# 이미지의 색공간을 BGR에서 RGB로 변환한다.mediapipe는 RGB이미지를 사용하기 때문
            result = hands.process(img)#손을 감지하는 함수
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)#이미지를 다시 BGR로 돌림. opencv는 BGR을 사용하기 때문

            if result.multi_hand_landmarks is not None:# 추출한 이미지의 위치, 관절 정보를 매핑해줌
                for res in result.multi_hand_landmarks:
                    joint = np.zeros((21, 4))
                    for j, lm in enumerate(res.landmark):
                        joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

                    # Compute angles between joints
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    v = v2 - v1 # [20, 3]
                    # Normalize v
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    # Get angle using arcos of dot product
                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

                    angle = np.degrees(angle) # Convert radian to degree

                    angle_label = np.array([angle], dtype=np.float32)
                    angle_label = np.append(angle_label, idx)

                    d = np.concatenate([joint.flatten(), angle_label])

                    data.append(d)

                    mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            cv2.imshow('img', img)
            if cv2.waitKey(1) == ord('q'):
                break

        data = np.array(data)
        print(action, data.shape)
        np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

        # Create sequence data
        full_seq_data = []
        for seq in range(len(data) - seq_length):
            full_seq_data.append(data[seq:seq + seq_length])

        full_seq_data = np.array(full_seq_data)
        print(action, full_seq_data.shape)
        np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    break

KeyboardInterrupt: 